## ThD Cases

In [ ]:
import os, sys, shutil
import numpy as np
from shutil import rmtree, copy
from matplotlib import pyplot as plt
from matplotlib import gridspec, cm
from PIL import Image, ImageDraw, ImageFont

# directory to the aspect Lab
ASPECT_LAB_DIR = os.environ['ASPECT_LAB_DIR']
RESULT_DIR = os.path.join(ASPECT_LAB_DIR, 'results')

sys.path.append(os.path.join(ASPECT_LAB_DIR))

import shilofue.ThDSubduction0.Cases as ThDCases
import shilofue.PlotCase as PlotCase
import shilofue.ThDSubduction0.PlotCase as ThDPlotCase
import shilofue.ThDSubduction0.VtkPp as ThDVtkPp
from shilofue.PlotCombine import PlotCombineExecute
from shilofue.ThDSubduction0.PlotVisit import PREPARE_RESULT_OPTIONS

#### System path

The path to the project directory

    local_TwoDSubduction_dir

The path to the project directory on server

    remote_TwoDSubduction_dir

In [ ]:
local_ThDSubduction_dir = "/mnt/lochy0/ASPECT_DATA/ThDSubduction"
remote_ThDSubduction_dir = "peloton:/group/billengrp-mpi-io/lochy/ThDSubduction"

assert(os.path.isdir(local_ThDSubduction_dir))

#### case name

case name (relative path to local_TwoDSubduction_dir)

    case_name

In [ ]:
# case_name = "EBA_2d_consistent_7/eba3d_width61_c22_AR4"
# case_name = "EBA_2d_consistent_8/eba3d_width61_c22_AR4"
# case_name = "EBA_2d_consistent_8_1/eba3d_width61_c22_AR4"
# case_name = "EBA_2d_consistent_8_2/eba3d_width61_c22_AR4"
# case_name = "EBA_2d_consistent_8_3/eba3d_width61_c22_AR4"
# case_name = "EBA_2d_consistent_8_2/eba3d_width61_c23_AR4"
# case_name = "EBA_2d_consistent_8_2/eba3d_width61_c22_AR4_cd150"
# case_name = "EBA_2d_consistent_8_4/eba3d_width61_c22_AR4"
# case_name = "EBA_2d_consistent_8_4/eba3d_width61_c22_AR4_old_rheology"
# case_name = "EBA_2d_consistent_8_4/eba3d_width61_c22_AR4_old_rheology_old_density"
# case_name = "EBA_2d_consistent_8_4/eba3d_width61_c22_AR4_old_density"
# case_name = "EBA_2d_consistent_8_5/eba3d_width61_c22_AR4_fix_density"

# in folder EBA_2d_consistent_8_6
# case_name = "EBA_2d_consistent_8_6/eba3d_width51_c22_AR4"
# case_name = "EBA_2d_consistent_8_6/eba3d_width61_c22_AR4"
# case_name = "EBA_2d_consistent_8_6/eba3d_width61_c22_AR3"
# case_name = "EBA_2d_consistent_8_6/eba3d_width61_c23_AR4"
# case_name = "EBA_2d_consistent_8_6/eba3d_width80_c22_AR4"
# case_name = "EBA_2d_consistent_8_6/eba3d_width80_c22_AR4_yd300"
# case_name = "EBA_2d_consistent_8_7/eba3d_width80_bw2000_sw500_c22_AR4"
case_name = "EBA_2d_consistent_8_7/eba3d_width80_bw8000_sw2000_c22_AR4"

local_dir = os.path.join(local_ThDSubduction_dir, case_name)
remote_dir = os.path.join(remote_ThDSubduction_dir, case_name)

#### Plot Linear results and prepare the scripts for visualization

The next block will generate plots of run time, solver, statistics, etc.
It will also compose a script to run in paraview / visit.

range of time to plot, when set to None, the full range is covered

    time_range

interval of time to plot, when set to None, every step is covered

    time_interval

step

Reset one of the options in the script to True in order to generate plot

    RUN_FULL_SCRIPT=False
    CROSS_SECTION_DEPTH=False
    PLOT_ISOVOLUME_WITH_STREAMLINE=False
    PLOT_Y_SLICES=False

In [ ]:
time_range = None
time_interval = None
# turn on plot_axis if I want to save a complete result
# turn off if I want to prepare for figures in a paper
plot_axis = True
step = [14]; slices=None # specify steps
# step = "auto"; slices=3  # auto-figure out the steps, take the numebr of slices
max_velocity = -1.0  # rescale the color for velocity


PlotCase.PlotCaseRun(local_dir, time_range=time_range, run_visual=False,\
        time_interval=time_interval, visualization="paraview", last_step=1)
plt.close() # plot won't show up below

Visit_Options = ThDPlotCase.PlotCaseRun(local_dir, time_range=time_range, run_visual=False,\
        time_interval=time_interval, visualization="paraview", step=step, plot_axis=plot_axis, max_velocity=max_velocity)
plt.close() # plot won't show up below

### Visualization, pt 1 : run paraview

run the following command with the script generated in the last block:

    paraview --script {paraview scripts}


#### adjust camera, run the following in Tools -> Python Script Editor

1. Upper mantle view:

```

# get active view
#### import the simple module from the paraview
from paraview.simple import *
#### disable automatic camera reset on 'Show'
paraview.simple._DisableFirstRenderCameraReset()

# get active view
renderView1 = GetActiveViewOrCreate('RenderView')

#-----------------------------------
# saving camera placements for views

# current camera placement for renderView1
renderView1.InteractionMode = '2D'
renderView1.CameraPosition = [0.0, 5.6e5, 2.5e7]
renderView1.CameraFocalPoint = [0.0, 6e6, 0.0]
renderView1.CameraParallelScale = 4.5e5

```

### Plot Slab Morphology

First analyze slab morphology (first cell).
Then plot slab morphology (second cell).

Interval to analyze.
0.5e6 is tested to give the best results in terms of capturing the trending and smearing out minor spikes.

    time_interval

In [ ]:
# the basic options to configure the slab envelop
slab_envelop_interval_y = 20e3  # Interval along x axis to sort out the trench locations
slab_envelop_interval_z = 20e3  # Interval along z axis to sort out the trench locations
slab_shallow_cutoff = 40e3  # Minimum depth along z axis to sort out the trench locations

In [ ]:
# generate the command for the slab envelop and run in terminal
crust_only = 1  # If we only use the crustal composition to sort out the trench locations
time_interval = 1e6 # The interval for trench position and velocities

print("python -m shilofue.ThDSubduction0.VtkPp morph_case -i %s -ti %.4e --slab_envelop_interval_y %.4e --slab_envelop_interval_z %.4e -ssc %.4e -co %d"\
        % (local_dir, time_interval, slab_envelop_interval_y, slab_envelop_interval_z, slab_shallow_cutoff, crust_only))

In [ ]:
# plot the ternch velocity
time_interval = 1e6

SlabPlot = ThDVtkPp.SLABPLOT('slab')
SlabPlot.PlotTrenchPosition(local_dir, time_interval=time_interval)
# SlabPlot.PlotMorph(local_dir, save_pdf=True)

plt.close()

In [ ]:
# plot the trench velocities in episodes
time_interval = 1e6

SlabPlot =  ThDVtkPp.SLABPLOT('slab')
episodes = [[0, 6], [5, 11], [10, 30]]
SlabPlot.PlotTrenchPositionEpisodes(local_dir, episodes, time_interval=time_interval)

# plt.close()

#### Functionalities to extract the slab surface in a cross section

In [ ]:
# Here I don't assign an additional crust_only field, thus I am using both sp_lower and sp_upper composition
# TODO: omit the intermediate results to not interfere with the previous section
vtu_snapshot = 100 + 4 # This is the snapshot = step + adaptive refinement

print("python -m shilofue.ThDSubduction0.VtkPp cross_section_at_depth -i %s -vss %d --slab_envelop_interval_y %.4e --slab_envelop_interval_z %.4e -ssc %.4e"\
        % (local_dir, vtu_snapshot, slab_envelop_interval_y, slab_envelop_interval_z, slab_shallow_cutoff))

In [ ]:
# plot the cross section of the slab surface at a given depth

depth=150e3
slab_surface_file="/home/lochy/ASPECT_PROJECT/aspectLib/.test/test_ThDSubduction_VtkPp/test_extract_slab_cross_section_at_depth/vtk_outputs/slab_surface_00144_d150.00km.txt"
assert(os.path.isfile(slab_surface_file))
data = np.loadtxt(slab_surface_file)
xs = data[:, 0]
ys = data[:, 1]
zs = data[:, 2]

fig, ax = plt.subplots()
ax.plot(xs/1e3, ys/1e3, "*")
ax.set_xlabel("X (km)")
ax.set_ylabel("Y (km)")
ax.set_title("Depth = %.2f km" % (depth/1e3))
ax.axis("equal")

#### Combine results for one case

I take the steps from the previous block where the paraview script is generated.

In [ ]:
# define the resize by width function to work with image.resize
resize_by_width = lambda size, width: (int(width), int(1.0 * size[1] *  width / size[0]))

# todo_comb
img_dir = os.path.join(local_dir, "img")
pv_output_dir = os.path.join(img_dir, "pv_outputs")
morphology_dir = os.path.join(img_dir, "morphology")
assert(os.path.isdir(pv_output_dir))
ns_image_path = os.path.join(img_dir, "newton_solver_history.png")
assert(os.path.isfile(ns_image_path))

# new image
# first initiate a new image
# The work flow of the Image module includes
# first openingthe figure and then extend that by
# opening new figures and paste them on the first one.
# One additional operation is appending text on the figure, this requries two additional packages
# ImageFont and ImageDraw
new_image_path = os.path.join(local_dir, "img", "case_combined.pdf")
image_size = (2700, 10000) # width, height
# image_size = (2700, int(h_last)) # width, height, do this if you want the exact height
new_image = Image.new('RGB',image_size,(250,250,250))
h_last = 0.0
h_interval = 100

# 1. paste the output from the new solver
# The figure is resized by the width of the canvas
ns_image = Image.open(ns_image_path)
ns_image = ns_image.resize(resize_by_width(ns_image.size, new_image.size[0]))
new_image.paste(ns_image, (0, 0))
h_last += ns_image.size[1]

# 2. paste the slab morphology plot
morphology_trench_image_path = os.path.join(morphology_dir, "trench_history.png")
morphology_velocity_image_path = os.path.join(morphology_dir, "trench_velocities.png")
assert(os.path.isfile(morphology_trench_image_path))
assert(os.path.isfile(morphology_velocity_image_path))
morphology_trench_image = Image.open(morphology_trench_image_path)
morphology_velocity_image = Image.open(morphology_velocity_image_path)
morphology_trench_image = morphology_trench_image.resize(resize_by_width(morphology_trench_image.size, new_image.size[0]/2.0))
morphology_velocity_image = morphology_velocity_image.resize(resize_by_width(morphology_velocity_image.size, new_image.size[0]/2.0))
new_image.paste(morphology_trench_image, (0, int(h_last+h_interval)))
new_image.paste(morphology_velocity_image, (int(new_image.size[0]/2.0), int(h_last+h_interval)))
h_last += max(morphology_trench_image.size[1], morphology_velocity_image.size[1])

# 3. paste the outputs from paraview
# Note "Visit_Options" is from a previous section where these plots are generated
steps = Visit_Options.options['GRAPHICAL_STEPS']
fnt0 = ImageFont.truetype("Pillow/Tests/fonts/FreeMono.ttf", int(h_interval / 3.0 * 2.0))
for step in steps:
    _time = step * 0.1e6
    print(_time) # debug
    # a. plot of viscosity 
    vis_center_image_path = os.path.join(pv_output_dir, "slice_trench_center_y_viscosity_t%.4e.png" % _time)
    assert(os.path.isfile(vis_center_image_path))
    vis_center_image = Image.open(vis_center_image_path)
    # print(vis_image.size)
    vis_center_image = vis_center_image.resize(resize_by_width(vis_center_image.size, new_image.size[0]/2.0))
    new_image.paste(vis_center_image, (0, int(h_last) + h_interval))
    # b. plot of the whole mantle
    vis_edge_image_path = os.path.join(pv_output_dir, "slice_trench_edge_y_viscosity_t%.4e.png" % _time)
    assert(os.path.isfile(vis_edge_image_path))
    vis_edge_image = Image.open(vis_edge_image_path)
    vis_edge_image = vis_edge_image.resize(resize_by_width(vis_edge_image.size, new_image.size[0]/2.0))
    new_image.paste(vis_edge_image, (int(np.ceil(new_image.size[0]/2.0)), int(h_last) + h_interval))
    h_last1 = h_last + vis_edge_image.size[1] + 0.5 * h_interval
    # c. cross section along z direction and plot the flow filed
    flow_100_image_path = os.path.join(pv_output_dir, "slice_100.0km_t%.4e.png" % _time)
    flow_200_image_path = os.path.join(pv_output_dir, "slice_200.0km_t%.4e.png" % _time)
    flow_100_image =  Image.open(flow_100_image_path)
    new_image.paste(flow_100_image, (0, int(h_last1)))
    flow_200_image =  Image.open(flow_200_image_path)
    new_image.paste(flow_200_image, (int(np.ceil(new_image.size[0]/2.0)), int(h_last1)))
    # d. plot a sub-title, including the time
    d = ImageDraw.Draw(new_image)
    d.text((int(new_image.size[0]/2.0), h_last), "t = %.2f Ma" % (_time/1e6), font=fnt0, fill=(0, 0, 0))
    h_last = h_last1 + flow_100_image.size[1] + h_interval

# save to a new figure
new_image.save(new_image_path)

In [ ]:
print(vis_image_path)

### Pick Timesteps

Pick in WebPlotDigitizer the time steps of:

1. Slab tip reaching 660 ($t_{660}$).
2. First peak of trench motion after $t_{660}$ ($t_{p1}$), and trench velocity
3. Second peak of trench motion after $t_{660}$ ($t_{p2}$), and trench velocity

### Visualization, pt 2 : save plots at these steps

Navigate to the paraview_scripts/slab.py file and change the steps.

### Plot Morphology Extra : Compare Morphology

Compare the slab morphology from multiple cases.

In [ ]:
case_name1 = "EBA_CDPT9/eba_cdpt_SA80.0_OA40.0_pc_mei_gr10_cf25GPa_PT1800_1"

json_option =\
{
    "_comment" : "This is configuration for combining results of time run (time, step, dofs ...)",
    "case_root": local_TwoDSubduction_dir,
    "cases": [case_name, case_name1],
    "output directory": {
        "relative": 1,
        "path": "plot_combine"
    },
    "width": -1.0,
    "time range": [0.0, 60e6],
    "trench position range": [-200e3, 200e3],
    "slab depth range": [0e3, 1800e3]
}

# print("json_option: ", json_option) # debug
PlotCombineExecute(TwoDVtkPp.PLOT_COMBINE_SLAB_MORPH, TwoDVtkPp.PC_MORPH_OPT, "slab_morph", json_option)

#### Make Animation

First generate the script for visualizations.
The time interval of visualizations for animation could be set through

    time_interval = 0.5e6

In [ ]:
# todo_ani

# turn on plot_axis if I want to save a complete result
# turn off if I want to prepare for figures in a paper
plot_axis = True
time_interval = 1e6
max_velocity = -1.0  # rescale the color for velocity

ThDPlotCase.PlotCaseRun(local_dir, time_range=None, run_visual=False,\
        time_interval=time_interval, visualization="paraview", step=None, plot_axis=plot_axis, max_velocity=-1.0)
plt.close() # plot won't show up below

_src = os.path.join(ASPECT_LAB_DIR, "files/ThDSubduction/bash_scripts/make_animation_paraview.sh")
_dst = os.path.join(local_dir, "paraview_scripts")
assert(os.path.isfile(_src))
shutil.copy(_src, _dst)

# replot the results of the morphology
# ThDPlotCase.PlotMorphAnimeCombined(local_dir, time_interval=time_interval)
# plt.close()

In [ ]:
# TODO: fix the flots and combine that into animation

SlabPlot = ThDVtkPp.SLABPLOT('slab')
vtu_step_list=[0, 15, 28]
this_vtu_step=200
SlabPlot.PlotTrenchPositionAnimation(local_dir, vtu_step_list, this_vtu_step, time_interval=1e6)

Then run scripts with

    pvpython slab.py

Note this will generate the visualizations for all the steps

Next, make the animation.

Assign the duration for each frame:

    duration = 0.2

By default, we remove the previous results and generate new results

    remove_old_results = True

Two default files are imbeded with operations for chunk and box geometry.
These operations will be performed on each figure to make the final animation.

In [ ]:
# this two are the json files for the order of options to do in imageio
# following the options defined in this two files, the results would be a combination of result for
# one single computation step.
time_interval = 1e6
duration = 0.2 # time for each frame
remove_old_results = True

default_box = os.path.join(ASPECT_LAB_DIR, "files", "ThDSubduction", "figure_step_template_box_04262024.json")

# remove old results
if remove_old_results:
    temp_dir = os.path.join(local_dir, "img", "pv_outputs", "temp")
    if os.path.isdir(temp_dir):
        rmtree(temp_dir)

pr_script = ThDPlotCase.PrScriptToUse(local_dir, default_box)
Plotter = ThDPlotCase.PLOTTER(PREPARE_RESULT_OPTIONS, [ThDPlotCase.PlotCaseRun]) # note we don't want to replot things here
PlotCase.AnimateCaseResults(Plotter.PlotPrepareResultStep, local_dir, pr_script, time_interval=time_interval, duration=duration)